In [12]:
from coati.common.periodic_table import XY_ONE_HOT_FULL
len(XY_ONE_HOT_FULL(1))

28

In [13]:
import torch
"""
atoms: batch X max_n_atom long tensor of atomic numbers.
coords: node coordinates.
"""
# prefer C2H6O and CH4 molecules as examples
atoms = torch.tensor([[6, 6, 8, 1, 1, 1, 1, 1, 1], [6, 1, 1, 1, 1, 0, 0, 0, 0]])
ans = atoms.tolist()
nodes = torch.tensor(
                    [[XY_ONE_HOT_FULL(int(atom)) for atom in mol] for mol in ans],  #transform atomic number into atomic table one-hot(x = 18, y = 10)
                    dtype=torch.float32,
                    device=atoms.device,
                    requires_grad=False,
                )

## 模型运作模式
### 1. 输入与编码（Encoding）
#### SMILES/SELFIES 编码：
使用 TrieTokenizer 对 SMILES 或 SELFIES 字符串进行分词。
RotarySmilesTransformer 捕捉序列中的长距离依赖关系，生成嵌入表示。
e3gnn_clip 图神经网络处理分子的图结构，提取图特征。

#### 坐标与原子信息编码：
e3gnn_clip 结合原子类型和空间坐标，生成节点特征。
e_gcl_sparse 等变图卷积层负责消息传递和特征更新，保持空间等变性。

#### eclouds 编码（可选）：
通过 ecloud_encoder 将 eclouds 信息嵌入到潜在空间，增强模型的表示能力。

### 2. 网络架构
#### 编码器（Encoders）：
图神经网络（e3gnn_clip）：提取分子的图结构特征，捕捉原子和键的关系。
Transformer 编码器（RotarySmilesTransformer）：处理 SMILES/SELFIES 序列，生成序列嵌入。
融合层：
将图特征和序列特征通过 smiles_to_clip 或 point_to_clip 进行投影和融合，形成统一的潜在表示。
#### 回归网络（regression）：
使用 FCResNet 全连接残差网络，从融合后的特征中提取高层次表示，用于预测分子属性。

### 3. 输出与预测
#### 分子属性预测：
回归网络输出分子的各类化学属性，如活性值、毒性等。
使用适当的损失函数（如 MSE）进行训练和优化。
#### 分子生成（可选）：
利用编码后的潜在表示，通过 Transformer 解码生成新的 SMILES 或 SELFIES 字符串，实现分子生成。
支持条件生成，通过注入特定嵌入（eclouds）控制生成分子的性质。

### 数据流动示意图
```
输入分子数据（SMILES/SELFIES + 坐标）
          │
          ▼
  分子表征编码（SMILES/SELFIES 令牌化）
          │
          ▼
  Transformer 编码（RotarySmilesTransformer）
          │
          ▼
   图神经网络编码（e3gnn_clip）
          │
          ▼
  特征融合与投影（smiles_to_clip / point_to_clip）
          │
          ▼
   回归网络（FCResNet）
          │
          ▼
   分子属性预测输出
```

In [14]:
import coati
import coati.common
import coati.common.periodic_table

in_node_nf = coati.common.periodic_table.XY_ONE_HOT_FULL(1)
print(in_node_nf)

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
